In [1]:
%matplotlib inline
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pylab
from PIL import Image

from torch import nn
from torch.autograd import Variable
from torchvision import models, transforms
import torch
import spectroUtils as sUtils

('track', 'tags')
('album', 'tags')
('artist', 'tags')
('track', 'genres')
('track', 'genres_all')


/Users/iaakhter/Documents/UBC/Winter22017/532L/DreamingInMusic/utils.py:218: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  'category', categories=SUBSETS, ordered=True)


## Scaling and Unscaling Spectrograms

In [2]:
# Find the min and max of spectrogram in all audio in directory
directory = "data/fma_small"
duration = 10 # duration of audio

minAmp, maxAmp = sUtils.minMax()
            
print ("minAmp", minAmp)
print ("maxAmp", maxAmp)

minAmp -80.0
maxAmp 1.9073486e-06
minAmp -80.0
maxAmp 1.9073486e-06


In [2]:
minAmp = -80.0
maxAmp = 1.9073486e-06
spectroSize = 224
trackId = 2
print(sUtils.getGenre(trackId))
sUtils.loadSpectro(trackId, spectroSize, minAmp, maxAmp)
    

Hip-Hop


(Variable containing:
 ( 0 ,.,.) = 
   131  171  179  ...   210  175  180
   172  186  190  ...   208  196  186
   194  199  198  ...   209  208  213
       ...         ⋱        ...      
    37   42   54  ...    58   58  106
    25   63   53  ...    54   66  106
    50   58   54  ...    49   51  106
 [torch.DoubleTensor of size 1x224x224],
 array([[-1.        +1.5099580e-07j,  1.        +0.0000000e+00j,
         -1.        +1.5099580e-07j, ..., -1.        +1.5099580e-07j,
         -1.        +1.5099580e-07j, -1.        +1.5099580e-07j],
        [-1.        -1.5099580e-07j, -0.51435715+8.5757607e-01j,
          0.03475438-9.9939591e-01j, ...,  0.5105215 -8.5986501e-01j,
          0.9926689 -1.2086546e-01j,  0.9867408 +1.6230398e-01j],
        [-1.        -1.5099580e-07j, -0.83196276-5.5483150e-01j,
          0.96983266+2.4377169e-01j, ..., -0.4854156 +8.7428355e-01j,
         -0.9999115 -1.3306852e-02j,  0.7704151 -6.3754255e-01j],
        ...,
        [ 1.        +1.0531130e-12j,  0.0

## Check whether scaling and unscaling spectrogram causes loss in audio quality

In [5]:
#Original audio
x, sr = librosa.load("data/fma_small/000/000002.mp3", sr=None, mono=True, duration = 10, offset = 0)
start, end = 0, 30
ipd.Audio(data=x[start*sr:end*sr], rate=sr)

In [6]:
trackId = 2
spectroSize = 224
scaledSpectro, phase, origSpectroSize = sUtils.loadSpectro(trackId, spectroSize, minAmp, maxAmp)
scaledSpectro =scaledSpectro.data[0]

# Get the power spectrogram from decibal version
backToSpectro = sUtils.unscaleSpectro(scaledSpectro, origSpectroSize, minAmp, maxAmp)
# Multiply power spectrogram with phase to get original complex spectrogram
backToSpectro = backToSpectro*phase

# Revert complex spectrogram to audio
n = len(x)
y_hat = librosa.istft(backToSpectro, length = n)
ipd.Audio(data=y_hat[start*sr:end*sr], rate=sr)

## Example Training

In [6]:
#https://github.com/srviest/char-cnn-pytorch/blob/master/model_CharCNN2D.py
class SpectroCNN(nn.Module):
    def __init__(self, output_size):
        super(SpectroCNN, self).__init__()    
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1)),
            nn.ReLU()
        ).double()

        self.maxpool1 = nn.MaxPool2d(kernel_size=(4, 4), stride=(1, 1))
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(222, 222, kernel_size=(2, 2), stride=(1,1)),
            nn.ReLU()
        ).double()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(200, 200), stride=(4, 4))
        
        self.fc1 = nn.Sequential(
            nn.Linear(1800, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        ).double()
        self.fc2 = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        ).double()
        self.fc3 =nn.Linear(1024, output_size).double()
        self.softmax = nn.LogSoftmax()
            # nn.LogSoftmax()

        # self.inference_log_softmax = InferenceBatchLogSoftmax()

    def forward(self, x):
        debug=False
        x = x.unsqueeze(1)
        if debug:
            print('x.size()', x.size())

        x = self.conv1(x)
        if debug:
            print('x after conv1', x.size())

        x = x.transpose(1,3)
        if debug:
            print('x after transpose', x.size())

        x = self.maxpool1(x)
        if debug:
            print('x after maxpool1', x.size())

        x = self.conv2(x)
        if debug:
            print('x after conv2', x.size())

        x = x.transpose(1,3)
        if debug:
            print('x after transpose', x.size())

        x = self.maxpool2(x)
        if debug:
            print('x after maxpool2', x.size())

        x = x.view(x.size(0), -1)
        if debug:
            print('Collapse x:, ', x.size())

        x = self.fc1(x).type(torch.DoubleTensor)
        if debug:
            print('FC1: ', x.size())

        x = self.fc2(x)
        if debug:
            print('FC2: ', x.size())

        x = self.fc3(x)
        if debug:
            print('x: ', x.size())

        x = self.softmax(x)
        # x = self.inference_log_softmax(x)

        return x



In [7]:
train_ids = [2,5,10,140,141,148,182,190,193,194]
tempCat = {'Pop':0, 'Rock':1, 'Hip-Hop':2}

def create_training(start, end):
    training_input = [loadSpectro(train_id).squeeze() for train_id in train_ids[start:end]
                      if trackExists(train_id)]

    # The output data is prepared by representing each output as a binary vector of categories
    training_output = []
    for i in range(start,min(len(train_ids),end)):
        if not(trackExists(train_ids[i])):
            continue
        genre = genres['title'][getGenreId(trackId)]
        training_vector = np.zeros(len(tempCat))
        training_vector[tempCat[genre]] = 1
        training_output.append(training_vector)
    training_output = Variable(torch.FloatTensor(training_output))
    
    return training_input, training_output

def train(model, learning_rate=0.0001, batch_size=1, epochs=1):
    """
    Training function which takes as input a model, a learning rate and a batch size.
  
    After completing a full pass over the data, the function exists, and the input model will be trained.
    """
    # Define the criterion and optimizer.
    criterion = nn.MultiLabelSoftMarginLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Keep track of the losses, for the purposes of plotting.
    train_losses = []

    # Determine number of minibatches
    num_iter = epochs * len(train_ids)//batch_size 
    for i in range(num_iter):
        print("Starting iteration: ", i)
        
        start_idx = i * batch_size % len(train_ids)
        
        training_input, training_output = create_training(start_idx, start_idx + batch_size)

        # Retrieve the next batch of training data.
        x = torch.stack(training_input)
        y = training_output.type(torch.DoubleTensor)

        # Forward pass
        y_pred = model(x)

        # Compute and print loss
        loss = criterion(y_pred, y)

        # Zero gradients, perform backwards pass and update model weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  
        
        del x,y,y_pred

        if i % 1 == 0:
            train_losses.append(loss.data[0])
            print(i, train_losses[-1])
        
    return train_losses 

# Finally train the model
numClasses = 3
model = SpectroCNN(numClasses)
model.train()
train(model)

Starting iteration:  0


/anaconda3/envs/myEnv2/lib/python3.6/site-packages/ipykernel_launcher.py:80: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0 5.423885960115509
Starting iteration:  1
1 1.8480728067984742
Starting iteration:  2
2 0.23472917934248896
Starting iteration:  3
3 0.23104908751567466
Starting iteration:  4
4 0.2310490601853169
Starting iteration:  5
5 0.23104906018531735
Starting iteration:  6
6 0.23104906018531504
Starting iteration:  7
7 0.23104908994478238
Starting iteration:  8
8 0.23104906018531504
Starting iteration:  9
9 0.23104906018531504


[5.423885960115509,
 1.8480728067984742,
 0.23472917934248896,
 0.23104908751567466,
 0.2310490601853169,
 0.23104906018531735,
 0.23104906018531504,
 0.23104908994478238,
 0.23104906018531504,
 0.23104906018531504]